# Mathematical Notation for Traffic Accident Hotspot Detection

Luke Zaruba, Bryan Runck

## LISA

![Local Moran's I](https://pro.arcgis.com/en/pro-app/latest/tool-reference/spatial-statistics/GUID-3638FD3C-9405-4A52-A3DE-5F209866C306-web.png)

[Source](https://pro.arcgis.com/en/pro-app/latest/tool-reference/spatial-statistics/h-how-cluster-and-outlier-analysis-anselin-local-m.htm)

[Paper](https://onlinelibrary.wiley.com/doi/epdf/10.1111/j.1538-4632.1995.tb00338.x)

## A-DBSCAN

![Pseudocode](https://ars.els-cdn.com/content/image/1-s2.0-S0094119019300944-gr11_lrg.jpg)

1. Split the dataset into two random subsets, one of them potentially smaller than the other (e.g. 10%/90%).

2. Run the original DBSCAN algorithm on one of the subsamples to obtain a set of cluster labels for each point in that subset.

3. For each point in the second subset, assign the label of the nearest point in the first subset.

4. Save the entire set of labels as a single candidate solution.

5. Repeat steps 1–4 a reasonably large amount of times (e.g. 1,000), obtaining several candidate solutions.

6. Align labels across candidate solutions so a given label represents the same cluster in each of the replications. This can be done in the following way:
    (a) Set the solution with most clusters as reference.
    (b) For each cluster in the remaining solutions, find the nearest cluster in the reference and assign its label. In this context, nearest can be expressed as the shortest distance between the centroids of the two clusters.
7. For each point in the dataset, obtain the most common label and the proportion of times across candidate solutions where that label is assigned.

8. If the most common label appears a proportion of times that is smaller than a desired threshold (e.g. 90%), label the observation as noise; otherwise assign the most common label as the label for that point.

[Source](https://www.sciencedirect.com/science/article/pii/S0094119019300944#fig0011)

## STCO
$ \boldsymbol{procedure} \text{ } Temporal \text{ } ADBSCAN(XYT, period, \epsilon , min\_pts\_pct, pct_{thin}, thr)\\
\qquad D \leftarrow \text{Dataset of point locations with timestamp } (XY \text{ coordinates and timestamp } T)\\
\qquad P \leftarrow \text{Unique Periods of type } period \text{ in }D \\
\qquad C \leftarrow \emptyset \\
\qquad \boldsymbol{for} \text{ p in P } \boldsymbol{do}\\
\qquad \qquad d \leftarrow D_T = p\\
\qquad \qquad ADBSCAN_p \leftarrow ADBSCAN(d_{XY}, \epsilon, \lvert d \rvert * min\_pts\_pct, pct_{thin}, thr)\\
\qquad \qquad Clusters_p \leftarrow GET \text{ } BOUNDARIES(ADBSCAN_p)\\
\qquad \qquad C \leftarrow C \cup Clusters_p
$

$ \boldsymbol{procedure} \text{ } Weighted \text{ } STCO(clusters)\\
\qquad D \leftarrow \text{Dataset of cluster polygons with period } p\\
\qquad P \leftarrow \text{Sorted Unique Periods in }D \\
\qquad W \leftarrow \emptyset \\
\qquad \boldsymbol{for} \text{ } index, value \text{ in P } \boldsymbol{do}\\
\qquad \qquad W \leftarrow W \cup {(value, index + 1)}\\
\qquad Sum_W \leftarrow \sum^{\lvert W \rvert}_{i=1} W_{i, 1}\\
\qquad \boldsymbol{for} \text{ } cluster \text{ in } D \text{ } \boldsymbol{do}\\
\qquad \qquad cluster_w \leftarrow W_{cluster_p, 1} / Sum_W\\
\qquad centroids \leftarrow UNARY \text{ } UNION(EXTERIOR(D))_{centroids}\\
\qquad WO \leftarrow SPATIAL \text{ } JOIN(centroids, D)\\
\qquad \boldsymbol{for} \text{ } overlap \text{ in } WO \text{ } \boldsymbol{do}\\
\qquad \qquad overlap_w \leftarrow \sum_{overlap \in WO} overlap_w
$

$ \boldsymbol{procedure} \text{ } Simple \text{ } STCO(clusters)\\
\qquad D \leftarrow \text{Dataset of cluster polygons}\\
\qquad centroids \leftarrow UNARY \text{ } UNION(EXTERIOR(D))_{centroids}\\
\qquad SO \leftarrow SPATIAL \text{ } JOIN(centroids, D)\\
\qquad \boldsymbol{for} \text{ } overlap \text{ in } SO \text{ } \boldsymbol{do}\\
\qquad \qquad overlap_w \leftarrow \lvert SO_{overlap} \rvert
$

## STCEC

$ \boldsymbol{procedure} \text{ } STCEC(clusters, significance)\\
\qquad D \leftarrow \text{Dataset of cluster polygons with period } p\\
\qquad P \leftarrow \text{Sorted Unique Periods in }D \\
\qquad centroids \leftarrow UNARY \text{ } UNION(EXTERIOR(D))_centroids\\
\qquad SO \leftarrow SPATIAL \text{ } JOIN(centroids, D)\\
\qquad \boldsymbol{for} \text{ } overlap \text{ in } SO \text{ } \boldsymbol{do}\\
\qquad \qquad overlap_{indexes} \leftarrow overlap_{indexes} \cup overlap_i\\
\qquad indexes \leftarrow Sort(Unique(SO_{overlap \text{ } indexes}))\\
\qquad length \leftarrow \lvert indexes \rvert\\
\qquad significance\_length \leftarrow length * significance\\
\qquad most\_recent\_index \leftarrow indexes[-1:]_{0}\\
\qquad second\_most\_recent\_index \leftarrow indexes[-2:-1]_{_0}\\
\qquad \boldsymbol{for} \text{ } overlap \text{ in } SO \text{ } \boldsymbol{do}\\
\qquad \qquad \boldsymbol{if} \text{ } \lvert overlap_{indexes} \rvert \geq significance\_length \text{ } \boldsymbol{then}\\
\qquad \qquad \qquad \boldsymbol{if} \text{ } most\_recent\_index \in overlap_{indexes} \text{ } \boldsymbol{then}\\
\qquad \qquad \qquad \qquad overlap_{class} \leftarrow PERSISTENT\\
\qquad \qquad \qquad \boldsymbol{else}\\
\qquad \qquad \qquad \qquad overlap_{class} \leftarrow HISTORICAL\\
\qquad \qquad \boldsymbol{else}\\
\qquad \qquad \qquad \boldsymbol{if} \text{ } \lvert overlap_{indexes} \rvert = 1 \text{ } \boldsymbol{then}\\
\qquad \qquad \qquad \qquad \boldsymbol{if} \text{ } most\_recent\_index \in overlap_{indexes} \text{ } \boldsymbol{then}\\
\qquad \qquad \qquad \qquad \qquad overlap_{class} \leftarrow NEW\\
\qquad \qquad \qquad \qquad \boldsymbol{else}\\
\qquad \qquad \qquad \qquad \qquad overlap_{class} \leftarrow SPORADIC\\
\qquad \qquad \qquad \boldsymbol{else}\\
\qquad \qquad \qquad \qquad \boldsymbol{if} \text{ } most\_recent\_index \in overlap_{indexes} \land second\_most\_recent\_index \in overlap_{indexes} \text{ } \boldsymbol{then}\\
\qquad \qquad \qquad \qquad \qquad overlap_{class} \leftarrow INTENSIFYING\\
\qquad \qquad \qquad \qquad \boldsymbol{else}\\
\qquad \qquad \qquad \qquad \qquad \boldsymbol{for} \text{ } i \text{ in } \lvert overlap_{indexes} \rvert \text{ } \boldsymbol{do}\\
\qquad \qquad \qquad \qquad \qquad \qquad consecutive \leftarrow overlap_{indexes, i} + 1 = overlap_{indexes, i + 1}\\
\qquad \qquad \qquad \qquad \qquad \boldsymbol{if} \text{ } consecutive = TRUE \text{ } \boldsymbol{then}\\
\qquad \qquad \qquad \qquad \qquad \qquad overlap_{class} \leftarrow DIMINISHING\\
\qquad \qquad \qquad \qquad \qquad \boldsymbol{else}\\
\qquad \qquad \qquad \qquad \qquad \qquad overlap_{class} \leftarrow OCCASIONAL
$